# Transfer Learning

In this notebook, you'll learn how to use pre-trained networks to solved challenging problems in computer vision. Specifically, you'll use networks trained on [ImageNet](http://www.image-net.org/) [available from torchvision](http://pytorch.org/docs/0.3.0/torchvision/models.html). 

ImageNet is a massive dataset with over 1 million labeled images in 1000 categories. It's used to train deep neural networks using an architecture called convolutional layers. I'm not going to get into the details of convolutional networks here, but if you want to learn more about them, please [watch this](https://www.youtube.com/watch?v=2-Ol7ZB0MmU).

Once trained, these models work astonishingly well as feature detectors for images they weren't trained on. Using a pre-trained network on images not in the training set is called transfer learning. Here we'll use transfer learning to train a network that can classify our cat and dog photos with near perfect accuracy.

With `torchvision.models` you can download these pre-trained networks and use them in your applications. We'll include `models` in our imports now.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

Most of the pretrained models require the input to be 224x224 images. Also, we'll need to match the normalization used when the models were trained. Each color channel was normalized separately, the means are `[0.485, 0.456, 0.406]` and the standard deviations are `[0.229, 0.224, 0.225]`.

In [2]:
data_dir = '/home/victor/data/.pytorch/Cat_Dog_data'

# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

We can load in a model such as [DenseNet](http://pytorch.org/docs/0.3.0/torchvision/models.html#id5). Let's print out the model architecture so we can see what's going on.

In [3]:
model = models.densenet121(pretrained=True)
model

/home/victor/anaconda3/lib/python3.7/site-packages/torchvision-0.2.1-py3.7.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

This model is built out of two main parts, the features and the classifier. The features part is a stack of convolutional layers and overall works as a feature detector that can be fed into a classifier. The classifier part is a single fully-connected layer `(classifier): Linear(in_features=1024, out_features=1000)`. This layer was trained on the ImageNet dataset, so it won't work for our specific problem. That means we need to replace the classifier, but the features will work perfectly on their own. In general, I think about pre-trained networks as amazingly good feature detectors that can be used as the input for simple feed-forward classifiers.

In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

In [5]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

With our model built, we need to train the classifier. However, now we're using a **really deep** neural network. If you try to train this on a CPU like normal, it will take a long, long time. Instead, we're going to use the GPU to do the calculations. The linear algebra computations are done in parallel on the GPU leading to 100x increased training speeds. It's also possible to train on multiple GPUs, further decreasing training time.

PyTorch, along with pretty much every other deep learning framework, uses [CUDA](https://developer.nvidia.com/cuda-zone) to efficiently compute the forward and backwards passes on the GPU. In PyTorch, you move your model parameters and other tensors to the GPU memory using `model.to('cuda')`. You can move them back from the GPU with `model.to('cpu')` which you'll commonly do when you need to operate on the network output outside of PyTorch. As a demonstration of the increased speed, I'll compare how long it takes to perform a forward and backward pass with and without a GPU.

In [6]:
import time

In [7]:
for device in ['cuda']:

    criterion = nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Move input and label tensors to the GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cuda; Time per batch: 0.005 seconds


You can write device agnostic code which will automatically use CUDA if it's enabled like so:
```python
# at beginning of the script
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# then whenever you get a new Tensor or Module
# this won't copy if they are already on the desired device
input = data.to(device)
model = MyModule(...).to(device)
```

From here, I'll let you finish training the model. The process is the same as before except now your model is much more powerful. You should get better than 95% accuracy easily.

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [8]:
## TODO: Use a pretrained model to classify the cat and dog images

<h1>Dense Net 121</h1>

In [9]:
model = models.densenet121(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

model.train()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
criterion = nn.NLLLoss()
model.to(device)

validate_every = 10

for e in range(2):
    running_loss = 0
    for i, (images, labels) in enumerate(trainloader):
        model.train()
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
            
        if i % validate_every == 0:
            print("[Epoch {0}] Training Loss = {1} ({2}/{3})".format(e, 
                                                                     running_loss / (i+1), 
                                                                     32 * (i+1), 
                                                                     len(trainloader) * 32))
            with torch.no_grad():
                model.eval()
                test_loss = 0
                accuracy = 0
                for i, (images, labels) in enumerate(testloader):
                    images, labels = images.to(device), labels.to(device)
                    outputs = model.forward(images)
                    loss = criterion(outputs, labels)
                    test_loss += loss.item()
                    ps = torch.exp(outputs)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                    
                print("[Epoch {0}] Test Loss = {1}".format(e, test_loss / len(testloader)))
                print("[Epoch {0}] Test Accuracy = {1}".format(e, accuracy / len(testloader)))

/home/victor/anaconda3/lib/python3.7/site-packages/torchvision-0.2.1-py3.7.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


[Epoch 0] Training Loss = 0.7294343709945679 (32/11264)
[Epoch 0] Test Loss = 0.4705995462834835
[Epoch 0] Test Accuracy = 0.807812511920929
[Epoch 0] Training Loss = 0.36823354932394897 (352/11264)
[Epoch 0] Test Loss = 0.08676576595753431
[Epoch 0] Test Accuracy = 0.9789062738418579
[Epoch 0] Training Loss = 0.30385878611178624 (672/11264)
[Epoch 0] Test Loss = 0.06517425074707717
[Epoch 0] Test Accuracy = 0.978515625
[Epoch 0] Training Loss = 0.25927473123996486 (992/11264)
[Epoch 0] Test Loss = 0.08369365651160479
[Epoch 0] Test Accuracy = 0.9691406488418579
[Epoch 0] Training Loss = 0.23861811310052872 (1312/11264)
[Epoch 0] Test Loss = 0.05067774085327983
[Epoch 0] Test Accuracy = 0.9828125238418579
[Epoch 0] Training Loss = 0.22346744803236981 (1632/11264)
[Epoch 0] Test Loss = 0.06748259398154914
[Epoch 0] Test Accuracy = 0.9750000238418579
[Epoch 0] Training Loss = 0.21481620727992448 (1952/11264)
[Epoch 0] Test Loss = 0.0638042530277744
[Epoch 0] Test Accuracy = 0.97773438692

[Epoch 1] Training Loss = 0.14553593242126053 (6752/11264)
[Epoch 1] Test Loss = 0.03902267967350781
[Epoch 1] Test Accuracy = 0.983203113079071
[Epoch 1] Training Loss = 0.14407117612558792 (7072/11264)
[Epoch 1] Test Loss = 0.047554627922363576
[Epoch 1] Test Accuracy = 0.9820312261581421
[Epoch 1] Training Loss = 0.14413862796379373 (7392/11264)
[Epoch 1] Test Loss = 0.03457892272854224
[Epoch 1] Test Accuracy = 0.9867187738418579
[Epoch 1] Training Loss = 0.14344868986747572 (7712/11264)
[Epoch 1] Test Loss = 0.03510875201318413
[Epoch 1] Test Accuracy = 0.985546886920929
[Epoch 1] Training Loss = 0.14232686424694688 (8032/11264)
[Epoch 1] Test Loss = 0.03614369088318199
[Epoch 1] Test Accuracy = 0.9847656488418579
[Epoch 1] Training Loss = 0.14306357278848975 (8352/11264)
[Epoch 1] Test Loss = 0.04596129227429628
[Epoch 1] Test Accuracy = 0.982421875
[Epoch 1] Training Loss = 0.14317800569248376 (8672/11264)
[Epoch 1] Test Loss = 0.04574923764448613
[Epoch 1] Test Accuracy = 0.983

<h2>Evaluate the Model</h2>

<h3>On Training Data</h3>

In [10]:
model.eval()

with torch.no_grad():
    running_loss = 0
    accuracy = 0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model.forward(images)
        ps = torch.exp(outputs)
        loss = criterion(outputs, labels)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        running_loss += loss.item()
        
print("Training Loss: {0}".format(running_loss / len(trainloader)))
print("Training Accuracy: {0}".format(accuracy / len(trainloader)))

Training Loss: 0.12004746511493894
Training Accuracy: 0.9488735198974609


<h3>On Test Data</h3>

In [11]:
with torch.no_grad():
    running_loss = 0
    accuracy = 0
    for i, (images, labels) in enumerate(testloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model.forward(images)
        ps = torch.exp(outputs)
        loss = criterion(outputs, labels)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        running_loss += loss.item()
        
print("Test Loss: {0}".format(running_loss / len(testloader)))
print("Test Accuracy: {0}".format(accuracy / len(testloader)))

Test Loss: 0.03452210831455886
Test Accuracy: 0.985546886920929


<h1>Resnet 50</h1>

In [15]:
model = models.resnet50(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 1024)),
                          ('relu1', nn.ReLU()),
                          ('dropout1', nn.Dropout(p=0.2)),
                          ('fc2', nn.Linear(1024, 512)),
                          ('relu2', nn.ReLU()),
                          ('dropout2', nn.Dropout(p=0.2)),
                          ('fc3', nn.Linear(512, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.fc = classifier

model.train()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)
criterion = nn.NLLLoss()
model.to(device)

for e in range(2):
    running_loss = 0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model.forward(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
            
        if i % validate_every == 0:
            print("[Epoch {0}] Training Loss = {1} ({2}/{3})".format(e, 
                                                                     running_loss / (i + 1), 
                                                                     32 * (i + 1), 
                                                                     len(trainloader) * 32))
#             with torch.no_grad():
#                 model.eval()
#                 test_loss = 0
#                 accuracy = 0
#                 for i, (images, labels) in enumerate(testloader):
#                     images, labels = images.to(device), labels.to(device)
#                     outputs = model.forward(images)
#                     loss = criterion(outputs, labels)
#                     test_loss += loss.item()
#                     ps = torch.exp(outputs)
#                     top_p, top_class = ps.topk(1, dim=1)
#                     equals = top_class == labels.view(*top_class.shape)
#                     accuracy += torch.mean(equals.type(FloatTensor))
                    
#                 print("[Epoch {0}] Test Loss = {1}".format(e, test_loss / len(testloader)))
#                 print("[Epoch {0}] Test Accuracy = {1}".format(e, accuracy / len(testloader)))

[Epoch 0] Training Loss = 0.7028878331184387 (32/11264)
[Epoch 0] Training Loss = 1.8482447700067 (352/11264)
[Epoch 0] Training Loss = 1.1348567236037481 (672/11264)
[Epoch 0] Training Loss = 0.9372382447604211 (992/11264)
[Epoch 0] Training Loss = 0.7870395888642567 (1312/11264)
[Epoch 0] Training Loss = 0.6783590641091851 (1632/11264)
[Epoch 0] Training Loss = 0.6044781495068894 (1952/11264)
[Epoch 0] Training Loss = 0.5428473325052732 (2272/11264)
[Epoch 0] Training Loss = 0.5014571793101452 (2592/11264)
[Epoch 0] Training Loss = 0.46627708557215364 (2912/11264)
[Epoch 0] Training Loss = 0.4350184211046389 (3232/11264)
[Epoch 0] Training Loss = 0.4201351102139499 (3552/11264)
[Epoch 0] Training Loss = 0.40109490633996064 (3872/11264)
[Epoch 0] Training Loss = 0.38509163553364406 (4192/11264)
[Epoch 0] Training Loss = 0.3693933787867955 (4512/11264)
[Epoch 0] Training Loss = 0.3560285232961178 (4832/11264)
[Epoch 0] Training Loss = 0.343759596972547 (5152/11264)
[Epoch 0] Training L

<h2>Evaluate the Model</h2>

<h3>On Training Data</h3>

In [16]:
model.eval()

with torch.no_grad():
    running_loss = 0
    accuracy = 0
    for i, (images, labels) in enumerate(trainloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model.forward(images)
        ps = torch.exp(outputs)
        loss = criterion(outputs, labels)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        running_loss += loss.item()
        
print("Training Loss: {0}".format(running_loss / len(trainloader)))
print("Training Accuracy: {0}".format(accuracy / len(trainloader)))

Training Loss: 0.12775485011198642
Training Accuracy: 0.9500720500946045


<h3>On Test Data</h3>

In [17]:
with torch.no_grad():
    running_loss = 0
    accuracy = 0
    for i, (images, labels) in enumerate(testloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model.forward(images)
        ps = torch.exp(outputs)
        loss = criterion(outputs, labels)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        running_loss += loss.item()
        
print("Test Loss: {0}".format(running_loss / len(testloader)))
print("Test Accuracy: {0}".format(accuracy / len(testloader)))

Test Loss: 0.04503030602354556
Test Accuracy: 0.985156238079071
